# Notacao IOB para NER (preprocess 2/2)

Notebook para incluir labels de entidades no esquema IOB para cada trecho de texto extraido em extract_text.ipynb

In [1]:
import pandas as pd
import re
import string

## Parte 1 - Carregamento dos dados

In [2]:
# data = pd.read_csv("verif_data.csv")
data = pd.read_csv("verif_dataV1.csv")
data['labels'] = ""

In [3]:
data.columns

Index(['REF_ANOMES', 'DATA_DODF', 'NUM_DODF', 'PAGINA_DODF', 'TIPO_DODF',
       'ATO', 'COD_EMPRESA', 'EMPRESA_ATO', 'COD_MATRICULA_ATO',
       'COD_MATRICULA_SIGRH', 'CPF', 'NOME_ATO', 'NOME_SIGRH', 'CARGO',
       'CLASSE', 'PADRAO', 'QUADRO', 'PROCESSO', 'FUND_LEGAL', 'text',
       'labels'],
      dtype='object')

Colunas a serem utilizadas: 'ATO', 'EMPRESA_ATO', 'COD_MATRICULA_ATO', 'CPF', 'NOME_ATO', 'CARGO', 'CLASSE',                                   'PADRAO', 'QUADRO', 'PROCESSO', 'FUND_LEGAL'

Colunas descartadas: 'Unnamed: 0', 'REF_ANOMES', 'DATA_DODF', 'NUM_DODF', 'PAGINA_DODF',
       'TIPO_DODF', 'COD_EMPRESA', 'COD_MATRICULA_SIGRH', 'NOME_SIGRH'

In [4]:
data = data.drop(columns=[ 
    'REF_ANOMES', 
    'DATA_DODF', 
    'NUM_DODF', 
    'PAGINA_DODF', 
    'TIPO_DODF', 
    'COD_EMPRESA', 
    'COD_MATRICULA_SIGRH', 
    'NOME_SIGRH'
])

In [5]:
data.columns

Index(['ATO', 'EMPRESA_ATO', 'COD_MATRICULA_ATO', 'CPF', 'NOME_ATO', 'CARGO',
       'CLASSE', 'PADRAO', 'QUADRO', 'PROCESSO', 'FUND_LEGAL', 'text',
       'labels'],
      dtype='object')

In [6]:
# print(data['FUND_LEGAL'][0], '\n\n')

# for row in range(len(data)):
#     for col in data.drop(columns=['text', 'labels']).columns:
#         if pd.notna(data[col][row]):
#             data[col][row] = str(data.loc[row][col]).replace('º', 'o')
# #             t.replace('º', 'o')
# #             data[col][row] = t

# for row in range(len(data)):
#     for col in data.drop(columns=['text', 'labels']).columns:
#         if pd.notna(data[col][row]):
#             if 'º' in str(data[col][row]):
#                 print(row, col)

# print(data['FUND_LEGAL'][0])

In [7]:
%%time
d = data.select_dtypes('object').copy()
d.fillna('nan', inplace=True)

d=d.applymap(
    lambda x: x.replace('º', 'o')
)
scalar = data.select_dtypes(exclude='object').copy()
d = scalar.join(d)
d.head(1)

CPU times: user 46.8 ms, sys: 1.51 ms, total: 48.3 ms
Wall time: 46 ms


,CPF,ATO,EMPRESA_ATO,COD_MATRICULA_ATO,NOME_ATO,CARGO,CLASSE,PADRAO,QUADRO,PROCESSO,FUND_LEGAL,text,labels
0,36868604149,CONCEDER APOSENTADORIA,"SECRETARIA DE ESTADO DE PLANEJAMENTO, ORCAMENT...",00317276,MARIA DE JESUS BARBOSA SANTOS,ANALISTA EM POLITICAS PUBLICAS E GESTAO GOVERN...,CLASSE ESPECIAL,PADRAO V,QUADRO DE PESSOAL DO DISTRITO FEDERAL,00410-00015650/2018-74,"NOS TERMOS DO ARTIGO 3o, INCISOS I, II E III, ...","CONCEDER, aposentadoria voluntaria integral, a...",


## Parte 2 - Funcoes para identificar entidades e anotar no padrao IOB

In [8]:
# Creates regex for the entity in data[col][idx]
def get_regex(data, col, idx):
    entity = str(data[col][idx])
    re_entity = ""
    if col == "COD_MATRICULA_ATO":
        while entity[0] == '0':
            entity = entity[1:]
        re_entity += '[0oO]*?'
    for i in entity:
        if (i >= 'a' and i <= 'z') or (i>='A' and i<='Z'):
            re_entity += f"[{i.lower()}{i.upper()}]" + "[-,.\s]*?"
        else:
            re_entity += i + "[-,.\s]*?"
    return re_entity

def get_regex_fast(s, col):
    entity = str(s)
    re_entity = ""
    if col == "COD_MATRICULA_ATO":
        entity = entity.lstrip('0')
        re_entity = '[0oO]*?'            
    re_entity += '(?i:{})'.format(
        "[-,.\s]*?".join(list(entity)+[''])
    )
    return re_entity


In [9]:
def IOBify_text(text, entities):
    labels = ["O" for _ in range(len(text.split()))]
    word_start_position = []
    number_of_word = []
    for i in range(len(text)):
        if (i == 0):
            if (text[i] == ' '):
                continue
            else:
                number_of_word.append(len(word_start_position))
                word_start_position.append(i)
        elif text[i] != ' ' and text[i-1] == ' ':
            number_of_word.append(len(word_start_position))
            word_start_position.append(i)
    i = 0
    for entity in entities:
        entity_begin = entity_end = -1
        # Find initial position of entity
        if entity[0] in word_start_position:
            entity_begin = word_start_position.index(entity[0])
        # Find final position of entity
        for pos, idx in zip(word_start_position, number_of_word):
            if pos > entity[1]:
                entity_end = idx-1
                break
        if entity_end == -1:
            entity_end = number_of_word[-1]
            
        if entity_begin != -1:
            for i in range(entity_begin, entity_end+1):
                if i==entity_begin:
                    labels[i] = "B-"+entity[2]
                else:
                    labels[i] = "I-"+entity[2]
    return labels
                
found_entities  = {col: 0 for col in data.drop(columns=['text', 'labels']).columns}
missed_entities = {col: [0, []] for col in data.drop(columns=['text', 'labels']).columns}

def find_entities(data, idx):
    list_entities = []
    for col in data.drop(columns=['text', 'labels']).columns:
        if pd.notna(data[col][idx]):
            re_entity = get_regex(data, col, idx)
            aux = re.search(re_entity, data['text'][idx])
            if aux:
                found_entities[col]  += 1
                list_entities.append([aux.span()[0], aux.span()[1], col])
            else:
                missed_entities[col][0] += 1
                missed_entities[col][1].append(idx)
    return list_entities

def find_entities_fast(data, idx):
    list_entities = []
    for col in data.drop(columns=['text', 'labels']).columns:
        if pd.notna(data[col][idx]):
            re_entity = get_regex_fast(data[col][idx], col)
            aux = re.search(re_entity, data['text'][idx])
            if aux:
                found_entities[col]  += 1
                list_entities.append([aux.span()[0], aux.span()[1], col])
            else:
                missed_entities[col][0] += 1
                missed_entities[col][1].append(idx)
    return list_entities


## Parte 3 - Identificando entidades e anotando-as

In [10]:
%%time
for row in range(len(data)):
    if row == 455 or row == 3372:
        continue
    if pd.notna(data['text'][row]):
        entities = find_entities_fast(data, row)
        entities.sort()
        labels = IOBify_text(data['text'][row], entities)
        s = ""
        for l in labels:
            s += l + ' '
        data['labels'][row] = s
#         data.loc['labels', row] = s
        
for col in found_entities:
    print(f"Found {col}: {found_entities[col]}", f"\t\t Missed {col}: {missed_entities[col][0]}")

<timed exec>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Found ATO: 5165 		 Missed ATO: 2
Found EMPRESA_ATO: 2204 		 Missed EMPRESA_ATO: 2963
Found COD_MATRICULA_ATO: 5155 		 Missed COD_MATRICULA_ATO: 12
Found CPF: 0 		 Missed CPF: 5167
Found NOME_ATO: 5157 		 Missed NOME_ATO: 10
Found CARGO: 5160 		 Missed CARGO: 6
Found CLASSE: 2381 		 Missed CLASSE: 39
Found PADRAO: 4871 		 Missed PADRAO: 6
Found QUADRO: 4903 		 Missed QUADRO: 47
Found PROCESSO: 5167 		 Missed PROCESSO: 0
Found FUND_LEGAL: 0 		 Missed FUND_LEGAL: 5166
CPU times: user 27.3 s, sys: 7.13 ms, total: 27.3 s
Wall time: 27.3 s


In [11]:
%%time
for row in range(len(data)):
    if row == 455 or row == 3372:
        continue
    if pd.notna(data['text'][row]):
        entities = find_entities(data, row)
        entities.sort()
        labels = IOBify_text(data['text'][row], entities)
        s = ""
        for l in labels:
            s += l + ' '
        data['labels'][row] = s
#         data.loc['labels', row] = s
        
for col in found_entities:
    print(f"Found {col}: {found_entities[col]}", f"\t\t Missed {col}: {missed_entities[col][0]}")

<timed exec>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Found ATO: 10330 		 Missed ATO: 4
Found EMPRESA_ATO: 4408 		 Missed EMPRESA_ATO: 5926
Found COD_MATRICULA_ATO: 10310 		 Missed COD_MATRICULA_ATO: 24
Found CPF: 0 		 Missed CPF: 10334
Found NOME_ATO: 10314 		 Missed NOME_ATO: 20
Found CARGO: 10320 		 Missed CARGO: 12
Found CLASSE: 4762 		 Missed CLASSE: 78
Found PADRAO: 9742 		 Missed PADRAO: 12
Found QUADRO: 9806 		 Missed QUADRO: 94
Found PROCESSO: 10334 		 Missed PROCESSO: 0
Found FUND_LEGAL: 0 		 Missed FUND_LEGAL: 10332
CPU times: user 29.1 s, sys: 3.75 ms, total: 29.1 s
Wall time: 29.1 s


In [12]:
msk=data.loc[:, 'text'].notna()
msk.head(1)

0    True
Name: text, dtype: bool

In [13]:
# data.text.map(str.lower, na_action='ignore')

In [14]:
data.apply(lambda x: x[1], axis=1, raw=True)

0       SECRETARIA DE ESTADO DE PLANEJAMENTO, ORCAMENT...
1       SECRETARIA DE ESTADO DO TRABALHO, DESENVOLVIME...
2           SERVICO DE LIMPEZA URBANA DO DISTRITO FEDERAL
3       SECRETARIA DE ESTADO DO TRABALHO, DESENVOLVIME...
4       SECRETARIA DE ESTADO DE PLANEJAMENTO, ORCAMENT...
                              ...                        
5511                        SECRETARIA DE ESTADO DE SAUDE
5512                        SECRETARIA DE ESTADO DE SAUDE
5513                        SECRETARIA DE ESTADO DE SAUDE
5514                        SECRETARIA DE ESTADO DE SAUDE
5515                        SECRETARIA DE ESTADO DE SAUDE
Length: 5516, dtype: object

In [15]:
data.head(1)

,ATO,EMPRESA_ATO,COD_MATRICULA_ATO,CPF,NOME_ATO,CARGO,CLASSE,PADRAO,QUADRO,PROCESSO,FUND_LEGAL,text,labels
0,CONCEDER APOSENTADORIA,"SECRETARIA DE ESTADO DE PLANEJAMENTO, ORCAMENT...",00317276,36868604149,MARIA DE JESUS BARBOSA SANTOS,ANALISTA EM POLITICAS PUBLICAS E GESTAO GOVERN...,CLASSE ESPECIAL,PADRAO V,QUADRO DE PESSOAL DO DISTRITO FEDERAL,00410-00015650/2018-74,"NOS TERMOS DO ARTIGO 3º, INCISOS I, II E III, ...","CONCEDER, aposentadoria voluntaria integral, a...",B-ATO I-ATO O O O O B-NOME_ATO I-NOME_ATO I-NO...


## Parte 4 - Corrigindo entidades nao identificadas

In [16]:
# Carregamento dos dados originais anotados
data_og = pd.read_csv('TCDF_data/Atos_Aposentadoria_validados.csv')

In [17]:
print("Entidades nao identificadas e as linhas do dataframe em que elas aparecem:\n")
for col in data.drop(columns=['text', 'labels']).columns:
    if len(missed_entities[col][1]) <= 100:
        print(f"Col:{col} -> {missed_entities[col][1]}") 

Entidades nao identificadas e as linhas do dataframe em que elas aparecem:

Col:ATO -> [3846, 4416, 3846, 4416]
Col:COD_MATRICULA_ATO -> [138, 422, 481, 506, 537, 571, 719, 761, 3199, 3610, 4449, 5512, 138, 422, 481, 506, 537, 571, 719, 761, 3199, 3610, 4449, 5512]
Col:NOME_ATO -> [201, 481, 571, 604, 2763, 5199, 5229, 5310, 5427, 5512, 201, 481, 571, 604, 2763, 5199, 5229, 5310, 5427, 5512]
Col:CARGO -> [138, 506, 571, 719, 4141, 5512, 138, 506, 571, 719, 4141, 5512]
Col:CLASSE -> [4, 13, 37, 59, 138, 506, 571, 947, 954, 1205, 1448, 1449, 1464, 1466, 1937, 1958, 2170, 2193, 2298, 2616, 2785, 3340, 3964, 4141, 4148, 4161, 4264, 4365, 4426, 4520, 4840, 4842, 4998, 5171, 5252, 5273, 5274, 5278, 5512, 4, 13, 37, 59, 138, 506, 571, 947, 954, 1205, 1448, 1449, 1464, 1466, 1937, 1958, 2170, 2193, 2298, 2616, 2785, 3340, 3964, 4141, 4148, 4161, 4264, 4365, 4426, 4520, 4840, 4842, 4998, 5171, 5252, 5273, 5274, 5278, 5512]
Col:PADRAO -> [138, 481, 571, 3199, 4449, 5512, 138, 481, 571, 3199, 444

In [18]:
row = 104
print(data['NOME_ATO'][row], "|||", data['COD_MATRICULA_ATO'][row], "|||", data_og['COD_MATRICULA_SIGRH'][row], "|||", data['PROCESSO'][row], "|||", data_og['NUM_DODF'][row], "|||", data_og['DATA_DODF'][row])
print(data['CLASSE'][row], "|||", data['PADRAO'][row], "|||", data['QUADRO'][row])
print()
print(data['text'][row])

DINA MARIA PIRES DE MIRANDA ||| 0025293X ||| 0025293X ||| 00361.00004000/201835 ||| 68 ||| 10/04/2018
CLASSE ESPECIAL ||| PADRAO V ||| QUADRO DE PESSOAL DO DISTRITO FEDERAL

CONCEDER Aposentadoria a DINA MARIA PIRES DE MIRANDA, matricula 25.293-X, no cargo de Auditor Fiscal de Atividades Urbanas, Classe Especial, Padrao V, do Quadro de Diario Oficial do Distrito Federal No 68, terca-feira, 10 de abril de 2018 PAGINA 20 Este documento pode ser verificado no endereco eletronico http://www.in.gov.br/autenticidade.html , pelo codigo 50012018041000020 Documento assinado digitalmente conforme MP n 2.200-2 de 24/08/2001, que institui a Infraestrutura de Chaves Publicas Brasileira - ICP-Brasil. Pessoal do Distrito Federal, nos termos do artigo 3o, incisos I, II, III, paragrafo unico da Emenda Constitucional no 47, de 06 de julho de 2005, combinado com o artigo 44, da Lei Complementar no 769, de 30 de junho de 2008. Processo SEI n o 00361.00004000/2018- 35


### Correcao manual de trechos de texto identificados erroneamente

Trechos de texto que nao condizem com as entidades anotadas sao corrigidos aqui

In [19]:
# Matricula(DONE) | NOME(DONE) | CARGO(DONE) | CLASSE(DONE) | PADRAO(DONE) | QUADRO() | 

# Fix row 138
data.loc['text', 138] = "CONCEDER aposentadoria voluntaria com proventos integrais a servidora a seguir nominada: MARIA AUXILIADORA PE- REIRA, matricula 101.008-5, processo SEI n 00070- 00011305/2018-23, no cargo de Au- xiliar de Desenvolvimento e Fiscalizacao Agropecuaria, Classe Unica, Padrao X, do Quadro de Pessoal do Distrito Federal, com fundamento no Art. 6, incisos I, II, III e IV, da EC n 41/ 2003, combinado com o artigo 2 da EC n 47/2005, c/c a Lei Complementar no 769/08."

# Fix row 481
data.loc['text', 481] = "CONCEDER APOSENTADORIA a IZAAC NEWTON DA SILVA, matricula 38.950-1, no Cargo de Professor de Educacao Basica, Padrao 22, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005. Processo 00080-00033434/2017-54."

# Fix row 506
data.loc['text', 506] = "CONCEDER aposentadoria voluntaria com proventos integrais ao servidor a seguir nominado: AMISAEL GONCALVES BI- NACETT, matricula 100.453-0, processo SEI n 00070-00013097/2018-05, no cargo de Auxiliar de Desenvolvimento e Fiscalizacao Agropecuaria, Classe Unica, Padrao X, do Quadro de Pessoal do Distrito Federal, com fundamento no Art. 3, incisos I, II, III e paragrafo unico da EC no 47/2005, c/c a LC no 769/2008."
data.loc['PROCESSO', 506] = "00070-00013097/2018-05"
data.loc['CARGO', 506] = "Auxiliar de Desenvolvimento e Fiscalizacao Agropecuaria"

# Fix row 537
data.loc['text', 506] = "CONCEDER APOSENTADORIA a VERA LUCIA FERREIRA DE SOUSA, matricula 40.787-9, no Cargo de Agente de Gestao Educacional/Servicos Gerais, Nivel 10, Padrao 02, Etapa III, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 3o da Emenda Constitucional no 47, de 06 de julho de 2005 e o Paragrafo unico do mesmo artigo. Processo 0 0 0 8 0 - 0 0 0 0 3 4 3 2 / 2 0 1 8 - 11 ."
data.loc['PROCESSO', 506] = "0 0 0 8 0 - 0 0 0 0 3 4 3 2 / 2 0 1 8 - 11"

#Fix row 571
data.loc['text', 571] = "CONCEDER APOSENTADORIA, nos termos do artigo 6o da Emenda Constitucional no 41/2003, combinado com o artigo 2o da Emenda Constitucional no 47/2005, combinados com o artigo 43, da Lei Complementar no 769, de 30/06/2008, a MARIA DE LOURDES SOUSA, matricula no 0143048-3, na Carreira de Assistencia Publica a Saude, no Cargo de Tecnico em Saude - TEC. LAB. HEMAT. E HEMOT, Primeira Classe, Padrao II, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: ADMC. Processo no 00060- 00247085/2017-21."

# Fix row 2763
data.loc['text', 2763] = "APOSENTAR MARIA ESTER BATISTA DE MEDEIROS, matricula 204.859-0, no Cargo de Professor de Educacao Basica, Padrao 19, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40, 1o, inciso I, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012. Processo 00040-00015628/2019-89."

# Fix row 5512
data.loc['text', 5512] = "APOSENTAR, nos termos do artigo 6o da Emenda Constitucional no 41/2003, combinado com o artigo 2o da Emenda Constitucional no 47/2005, combinados com o artigo 43, da Lei Complementar no 769, de 30/06/2008, a MARIA DE LOURDES SOUSA, matricula no 0143048-3, na Carreira de Assistencia Publica a Saude, no Cargo de Tecnico em Saude - TEC. LAB. HEMAT. E HEMOT, Primeira Classe, Padrao II, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: ADMC. Processo no 00060- 00247085/2017-21."

# Fix row 

### Correcao manual das entidades de MATRICULA

utilizando as matriculas SIGRH quando matriculas ATO nao sao encontradas

In [20]:
#Fix row 138
data.loc['COD_MATRICULA_ATO', 138] = data_og.iloc[138]['COD_MATRICULA_SIGRH']

#Fix row 422
data.loc['COD_MATRICULA_ATO', 422] = data_og.iloc[422]['COD_MATRICULA_SIGRH']

#Fix row 719
data.loc['COD_MATRICULA_ATO', 719] = data_og.iloc[719]['COD_MATRICULA_SIGRH']

# FIx row 761
data.loc['COD_MATRICULA_ATO', 761] = data_og.iloc[761]['COD_MATRICULA_SIGRH']

# Fix row 3199
data.loc['COD_MATRICULA_ATO', 3199] = "209.913-6"

# Fix row 3610
data.loc['COD_MATRICULA_ATO', 3610] = data_og.iloc[3610]['COD_MATRICULA_SIGRH']

# Fix row 4449
data.loc['COD_MATRICULA_ATO', 4449] = "42.299-1"

### Correcao manual das entidades de NOME

Diferenca no processamento de acentos e apostrofos entre o DODFminer e as anotacoes

In [21]:
# Fix row 201
data.loc['NOME_ATO', 201] = "ROSELI DE MOURA GONZALES D ALMEIDA"

# Fix row 604
data.loc['NOME_ATO', 604] = "MARIA FLORA CALVINO MARQUES"

# Fix row 5199
data.loc['NOME_ATO', 5199] = "LORIVANDA D ABADIA DOS SANTOS"

# Fix row 5229
data.loc['NOME_ATO', 5229] = "ELZA LUCIA MENDES MUNIZ"

# Fix row 5310
data.loc['NOME_ATO', 5310] = "MARIA D ARC PEREIRA"

# Fix row 5427
data.loc['NOME_ATO', 5427] = "MARCIA CRISTINA TOMAZ MULLER"

### Correcao manual das entidades de CARGO

Algumas anotacoes erroneas (apresentadas abaixo)

In [22]:
# Fix row 719
data.loc['CARGO', 719] = "agente de transito"

# Fix row 4141
data.loc['CARGO', 4141] = "Gestor em Politicas Publicas e Gestao Governamental"

### Correcao manual das entidades de CLASSE

Em grande parte diferenca entre numeros ordinais e suas escritas por extenso (1a classe == primeira classe)

In [26]:
for i in missed_entities['CLASSE'][1]:
    a = data.iloc[i]['CLASSE']
    a = a.split()
    s = ""
    if a[0].lower() == 'primeira':
        s = "1a Classe"
    elif a[0].lower() == 'segunda':
        s = "2a Classe"
    elif a[0].lower() == 'terceira':
        s = "3a Classe"
    elif a[0].lower() == 'quarta':
        s = "4a Classe"
    if s:
        data['CLASSE', i] = s
        
# Fix row 506
data.loc['CLASSE', 506] = "Classe Unica"

# Fix row 947
data.loc['CLASSE', 947] = "Classe Unica"

# Fix row 5512
data.loc['CLASSE', 5512] = "Classe Unica"

### Correcao manual das entidades de PADRAO

Entidades da classe PADRAO foram anotadas erroneamente (casos abaixo)

In [24]:
# Fix row 719
data.loc['CARGO', 719] = "agente de transito"

# Fix row 4141
data.loc['CARGO', 4141] = "Gestor em Politicas Publicas e Gestao Governamental"

### Correcao manual das entidades de QUADRO

Ainda nao sei qual o erro

## Final - Salvando os dados preprocessados

Salva o dataframe com os trechos de texto e os rotulos em arquivo csv para treinamento de modelos de NER

In [25]:
data.to_csv('labeled.csv', index=False)